# Description from kaggle:

**TODO: SPELLING**

**Problem:**

<i>"I have a friend who working in a small team that taking care of water pump of a small area far from big town, there are 7 system failure in last year. Those failure cause huge problem to many people and also lead to some serious living problem of some family. The team can't see any pattern in the data when the system goes down, so they are not sure where to put more attention.

Since I believe in using data to solve problem, I ask him to provide available sensor data and hope that someone here can help."</i>

**Data:**

<i>"The data are from all available sensor, all of them are raw value. Total sensor are 52 unit."</i>

Source: https://www.kaggle.com/nphantawee/pump-sensor-data


# Imports:

In [2]:
import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder,power_transform
from hmmlearn import hmm

from plotly.subplots import make_subplots
import plotly.graph_objects as go

# # %matplotlib notebook
# %matplotlib widget

# Load data:

In [3]:
#the first few lines of the csv:
with open('sensor.csv','r') as f:
    for i,line in enumerate(f):
        if i == 2: break
        print(line)
    

,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27,sensor_28,sensor_29,sensor_30,sensor_31,sensor_32,sensor_33,sensor_34,sensor_35,sensor_36,sensor_37,sensor_38,sensor_39,sensor_40,sensor_41,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,machine_status

0,2018-04-01 00:00:00,2.465394,47.092009999999995,53.2118,46.310759999999995,634.375,76.45975,13.41146,16.13136,15.567129999999999,15.053529999999999,37.2274,47.52422,31.11716,1.6813529999999999,419.5747,,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.674,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.98958,31.770832061767

In [ ]:
# load csv into pandas dataframe:
dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
df = pd.read_csv('sensor.csv',sep=',',parse_dates=['timestamp'], date_parser=dateparse)
df = df.drop(df.columns[0], axis=1)
df = df.set_index('timestamp')
print('df.shape',df.shape)
df.head()

In [ ]:
df.describe()

# Exploring the data:

### Investigating potential clusters with the correlation matrix:

In [ ]:
#Copied from: https://seaborn.pydata.org/examples/many_pairwise_correlations.html

# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
f.show()

 - Initially nothing majorly correlated, but there do seem to be groupings of sorts

### Visualize groupings seen in correlation matrix:

#### Functions:

In [ ]:
def generate_plotly_timeseries(df: pd.DataFrame, sensor_ids: range) -> go.Figure: 
    
    fig = make_subplots(rows=len(sensor_ids), cols=1,
                        shared_xaxes='columns',
                        subplot_titles=[f'sensor_{i:02d}' for i in sensor_ids],
                        specs=[[{"type": "scatter"}] for x in sensor_ids]
                       )
    
    for i in sensor_ids:
        
        sensor_name = f'sensor_{i:02d}'
        # plotly subplot row count can ponly start from 1
        if sensor_ids.start == 0:
            i+=1
        else:
            # +1 to start rowcount from 1
            i = (i % sensor_ids.start) + 1 
        
        if sensor_name in df.columns:
            
            data = df[sensor_name]
            fig.add_scatter(
                x=data.index,
                y=data,
                name=sensor_name,
                showlegend=True,
                row=i, col=1
            )
    
    #Adding broken status: 
    for date in df[df.machine_status=='BROKEN'].machine_status.index:
        fig.add_vline(x=date,
                      row='all',
                      line=dict(dash='dash',color='rgba(0, 0, 0, 0.6)', width=2)
                     )
    
    return fig

#### Generating sensor timeseries as a subplot:
- based on the groupings seen in the correlation matrix
- using plotly for interactive plots (runtime is about 3-5 minutes)

In [ ]:
sensor_ranges = [range(0,14),
                 range(14,37),
                 range(36,38),
                 range(38,52),
                ]
for sensor_ids in sensor_ranges:

    fig = generate_plotly_timeseries(df,sensor_ids)
    fig.update_layout(height=1200, width=1200, template='plotly_white')#, title_text="Side By Side Subplots")

    # Large interactive plotly charts using fig.show() can lead to excess ram consumption for jupyter, writing to html file instead.
    # fig.show()

    fig.write_html(f'./plotly_charts/sensor_timeseries_subplot_{sensor_ids.start}-{sensor_ids.stop-1}')


### Identify and drop sparse columns

- if there are missing values more than 1 pct of the time then drop those columns \
this might need to be tweeked, but lets see how it spans out:


In [ ]:
print('original df shape',df.shape)
x = df.isna().sum()

# if there are missing values more than 1 pct of the time then drop those columns
# this might need to be tweeked, but lets see how it spans out:
threshold = df.shape[0]*0.01
nan_cols = x[x>threshold].index
df = df.drop(labels=nan_cols,axis=1)

print('dropped cols: ', nan_cols.values)
print('df after dropping',df.shape)
print('Remaining missing data:')

for i,element in enumerate(zip(df.isna().sum().index,df.isna().sum())):
    idx,val = element
    if (i % 8) == 0 and i > 0:
        print()
    print(f'{idx}: {val}',end ='\t')

fig,ax=plt.subplots(1,1,figsize=(12,5))
df.isna().sum().plot(kind='bar',ax=ax)
ax.set_ylabel('# of missing entries')
fig.show()

### Standardize / normalize values of each timeseries:

In [ ]:
# df.fillna()

In [ ]:
## Optionally do it for all sensors:
#####
df.iloc[:,:-1] = StandardScaler().fit_transform(df.iloc[:,:-1])


### Misc:

In [ ]:
print(df.shape)
list(zip(df.isna().sum(),df.dtypes))

# Init data exploration:

In [ ]:
df.columns

### How are the labels distributed?

In [ ]:
display(df.machine_status.value_counts())
print('\n\nThe same in percentages :')
df.machine_status.value_counts(normalize=True)*100


 - Broken status is extremely rare it only occurs 0.003% of the time!
 - Initial thoughts are to try and visually demontrate how a broken status happens on a device and try and figure out a pattern.
 - I think a standard ML will struggle to identify broken status on such an unbalanced dataset

In [ ]:
df[df.machine_status == 'BROKEN'].isna().sum(axis=1)

 -  Whenever machine_status == 'BROKEN' there does seems to be incoming data!

### Are 'BROKEN' labels always followed by 'RECOVERING' and preceded by 'Normal'?

In [ ]:
idx = df[df.machine_status == 'BROKEN'].index

# dt = 2400
dt_m = 30
dt_p = 1000
df_labels= df[['machine_status']].copy()
print(df_labels.shape)

encoder = LabelEncoder().fit(df_labels[['machine_status']])
print(encoder.classes_)

df_labels['machine_status'] = encoder.transform(df_labels[['machine_status']])

df_labels.head()

fig,ax = plt.subplots(7,1,figsize=(10,8))
fig.tight_layout()

for i,x in enumerate(idx):    
    
    t_m_dt = x - datetime.timedelta(minutes = dt_m)
    t_p_dt = x + datetime.timedelta(minutes = dt_p)
    
    df_plot = df_labels.loc[t_m_dt:t_p_dt,['machine_status']].copy()

    ax[i].plot(df_plot,label=x)
    ax[i].axvline(idx[i],linestyle='--',color='r')
    ax[i].set_title(f'incident #{i} time evolution')
    ax[i].legend()
    ax[i].grid()

In [ ]:
shifted = df_labels.machine_status.shift(-1)
orig = df_labels.machine_status

condition = (orig !=2) & (shifted == 2)
np.where(condition,1,0).sum()

### How does a particular sensor behave around the machine_status == 'broken' timestamps?

In [ ]:

idx = df[df.machine_status == 'BROKEN'].index

dt = 200
sensor= 'sensor_04'
fig,ax = plt.subplots(7,1,figsize=(10,15))
fig.tight_layout()
for i,x in enumerate(idx):    
    
    t_m_dt = x - datetime.timedelta(minutes = dt)
    t_p_dt = x + datetime.timedelta(minutes = dt)
    
    df_plot = df.loc[t_m_dt:t_p_dt,[sensor]].copy()

    ax[i].plot(df_plot,label=x)
    ax[i].axvline(idx[i],linestyle='--',color='r')
    ax[i].set_title(f'incident #{i} on {sensor}')
    ax[i].legend()
    ax[i].grid()


    

### How does each sensor behave in relation to other sensors during a Broken status?
 

In [ ]:
# one sensor

idx = df[df.machine_status == 'BROKEN'].index

dt = 30
t_m_dt = idx[0] - datetime.timedelta(minutes = dt)
t_p_dt = idx[0] + datetime.timedelta(minutes = dt)

df_plot = df.loc[t_m_dt:t_p_dt,:].copy()
df_plot = df_plot.iloc[:,:-1]
df_plot.iloc[:,:] = StandardScaler().fit_transform(df_plot)
# display(df_plot.columns)

### Make plots:
# fig,ax = plt.subplots(nrows=7,ncols=7,figsize=(10,5))
fig,ax = plt.subplots(nrows=1,ncols=1,figsize=(10,5))
# fig.tight_layout()

sensor= 'sensor_01'
ax.plot(df_plot[[sensor]],'b-o',label= 'sensor')
ax.axvline(idx[0],linestyle='--',color='r',label = str(idx[0]))

rest = [x for x in df_plot.columns if x != sensor]
for s in rest:
    ax.plot(df_plot[[s]],'-',color='lightgrey',alpha=0.3)

#dummy for legend:
ax.plot([],[],'-',color='lightgrey',alpha=0.3,label='rest of sensors')

ax.set_title(sensor)
ax.legend(loc='upper left')



In [ ]:
# all sensors

idx = df[df.machine_status == 'BROKEN'].index
incident = idx[2]

dt = 120
t_m_dt = incident - datetime.timedelta(minutes = dt)
t_p_dt = incident + datetime.timedelta(minutes = dt)
# t_m_dt = incident - datetime.timedelta(hours = dt)
# t_p_dt = incident + datetime.timedelta(hours = dt)



df_plot = df.loc[t_m_dt:t_p_dt,:].copy()
df_plot = df_plot.iloc[:,:-1]
df_plot.iloc[:,:] = StandardScaler().fit_transform(df_plot)
# display(df_plot.columns)

### Make plots:
n_rows = 9
n_cols = 5

fig,ax = plt.subplots(nrows=n_rows,ncols=n_cols,figsize=(12,14), sharex=True, sharey=True)
# fig.tight_layout()

for i,sensor in enumerate(df_plot.columns):
    
    row= i // n_cols
    col= i % n_cols
#     print(row,col,i,sensor)

#     rest = [x for x in df_plot.columns if x != sensor]
#     for s in rest:
#         ax[row][col].plot(df_plot[[s]],'-',color='lightgrey',alpha=0.3)

    ax[row][col].plot(df_plot[[sensor]],'b.',label= f'{sensor} row{row},col{col} ',alpha=0.3)
    ax[row][col].axvline(incident,linestyle='--',color='r',label = 'ts of incident')
    ax[row][col].set_title(sensor)
    
    #dummy plot
    ax[row][col].plot([],[],'-',color='lightgrey',alpha=0.3,label='rest of sensors')
    ax[row][col].tick_params(labelrotation=45)

ax[-1][-1].tick_params(labelrotation=45)
fig.suptitle(f'Broken incident timestamp {idx[-1]} \n')
fig.tight_layout()


 - Similar to the correlation plot there seem to be underlying groupings in the sensors. For example the sensors 1-13 , 14-36/37 and 38-49 exhibit "similar" patterns.

### Plots:

In [ ]:
df.columns

#### Taking a look at an individual timeseries:

In [ ]:
plt.figure(figsize = (10,8))

sensor_name = 'sensor_04'
# sensor_name = 'sensor_41'
val = StandardScaler().fit_transform(df[[sensor_name]].values)
val = pd.Series(val.ravel()).fillna(0)
val = val.rolling(window=60,center=True).median()

print(df.index.shape)
print(val.shape)

plt.plot(df.index,val,label = sensor_name)

idx = df[df.machine_status == 'BROKEN'].index
for x in idx:
    plt.axvline(x,ymax=12,linestyle='--',color='r')
    
# idx = df[df.machine_status == 'RECOVERING'].index[5::30]
# for x in idx:
#     plt.axvline(x,ymax=12,linestyle='--',color='g')
    
m = pd.Series(val.ravel()).fillna(0).mean()
s = pd.Series(val.ravel()).fillna(0).std()
print(m,s)

plt.hlines(m-3*s, df.index[0], df.index[-1], colors='r', linestyles='--')
# plt.hlines(df.index,[ for x in range(len(val))],'--',label='3 sigma')
plt.legend()


#### FFT spectrum:

In [ ]:
# TODO
sensor_name = 'sensor_04'
# sensor_name = 'sensor_05'
# sensor_name = 'sensor_13'
sensor_data = df[sensor_name].copy().fillna(0).values
sensor_data -= np.mean(sensor_data)

fft_data = np.fft.rfft(sensor_data)#,norm='ortho')
freqs = np.fft.rfftfreq(len(sensor_data))
spectrum = np.abs(fft_data)

filt_index = np.argmax(fft_data) + 200
filtered_fft = fft_data[:]
filtered_fft[filt_index:] = 0
filt_signal = np.fft.irfft(filtered_fft)

fig,ax = plt.subplots(2,1,figsize=(12,8))
ax[0].plot(freqs,spectrum)
ax[0].set_xlim(-0.0001,0.005)
ax[1].plot(df.index,sensor_data)
ax[1].plot(df.index,filt_signal)

idx = df[df.machine_status == 'BROKEN'].index
for x in idx:
    plt.axvline(x,ymax=12,linestyle='--',color='r')

m= sensor_data.mean()
s= sensor_data.std()
ax[1].hlines(m-1*s, df.index[0], df.index[-1], colors='r', linestyles='--')

for axis in ax:
    axis.grid()
fig.show()

#### Testing spectral residual idea:

In [ ]:
#more on this: https://arxiv.org/pdf/1906.03821.pdf
# and here: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.125.5641&rep=rep1&type=pdf

plt.figure(figsize = (10,8))

sensor_name = 'sensor_04'
val = StandardScaler().fit_transform(df[[sensor_name]].values)
val = pd.Series(val.ravel()).fillna(0)
val = val.rolling(window=60,center=True).median()

# plt.plot(df.index,val,label = sensor_name)

idx = df[df.machine_status == 'BROKEN'].index
for x in idx:
    plt.axvline(x,ymax=12,linestyle='--',color='r')

#### Spectral residual:
sr_ts = df[sensor_name].copy().fillna(0).values
sr_ts -= sr_ts.mean()
print(sr_ts)

fft = np.fft.fft(sr_ts)
amp = np.abs(fft)
log_amp = np.log(amp)
phase = np.angle(fft)

win_size = 600
conv_amp = np.array([1/win_size for x in range(win_size)])
ma_log_amp = np.convolve(log_amp, conv_amp, 'same')

res_amp = log_amp - ma_log_amp
sr = np.abs(np.fft.ifft(np.exp(res_amp + 1j * phase)))

plt.plot(df.index,sr,label = 'spectral redisual')

# m = pd.Series(val.ravel()).fillna(0).mean()
# s = pd.Series(val.ravel()).fillna(0).std()
# print(m,s)

# plt.hlines(m-3*s, df.index[0], df.index[-1], colors='r', linestyles='--')
# # plt.hlines(df.index,[ for x in range(len(val))],'--',label='3 sigma')

plt.legend()

 - this transformation / spike detection looks promising ,but would require more time to investigate


#### Further plots:

In [ ]:
plt.figure(figsize = (10,8))

sensor_name = 'sensor_49'
val = StandardScaler().fit_transform(df[[sensor_name]].values)

kernel = np.array([-1,1])
val_diff = np.correlate(val.ravel(),kernel,mode='same')

plt.plot(df.index,val,label = sensor_name)
# plt.plot(df.index,val_diff,label = sensor_name + '_diff')

idx = df[df.machine_status == 'BROKEN'].index
for x in idx:
    plt.axvline(x,ymax=12,linestyle='--',color='r')
    
plt.legend()


In [ ]:
df.iloc[:,:-1].mean(axis=1)

In [ ]:
plt.figure(figsize = (16,8))
plt.plot(df.iloc[:,:-1].mean(axis=1))
idx = df[df.machine_status == 'BROKEN'].index
for x in idx:
    plt.axvline(x,ymax=12,linestyle='--',color='r')

# Simple Model

 - simple thresholding based on standard deviation of signal.
 - sensor #4 seems like an ideal candidate BROKEN status is nearly always followed by a 3*sigma swing in signal values.
 - split timeseries into 2 parts, train and test. Use train set to calculate standard deviation (sigma)
 - Locate crossings of the threshold (only do it when derivative is negative).
 - will generate many false positives, ie Model says it is broken, yet it is actually in recovery phase.
 - on the plus side few false negatives (hopefully)


 
     

## Train phase:

In [ ]:
def ThresholdCrossings(x):

    ## Detect threshold crossings:
    signal_sign_diff = np.diff(np.sign(signal_crossing))

    #zero crossings, only checking for positive to negative
    crossings = np.where(signal_sign_diff < 0,signal_sign_diff,0)
    crossing_idx = crossings.nonzero()[0]

    return crossing_idx

def filter_signal(sensor_data):
        
    fft_data = np.fft.rfft(sensor_data)#,norm='ortho')
    freqs = np.fft.rfftfreq(len(sensor_data))
    spectrum = np.abs(fft_data)

    filt_index = np.argmax(fft_data) + 200
    filtered_fft = fft_data[:]
    filtered_fft[filt_index:] = 0
    filt_signal = np.fft.irfft(filtered_fft)
    
    return filt_signal

In [ ]:
## Split train / test
split_idx = df.shape[0] // 2 

sensor_name = 'sensor_04'
signal = StandardScaler().fit_transform(df[[sensor_name]].values)
signal = pd.Series(signal.ravel()).fillna(0)

plt.figure(figsize=(10,8))

plt.plot(df.index[:split_idx],signal.iloc[:split_idx],label = sensor_name)

## Plot threshold
sigma_train = signal.std()
plt.hlines(-3*sigma_train, df.iloc[:split_idx].index[0], df.iloc[:split_idx].index[-1], colors='r', linestyles='--',label = 'threshhold')


## Detect threshold crossings:
signal_crossing = signal.iloc[:split_idx] + 3*sigma_train 
crossing_idx = ThresholdCrossings(signal_crossing)

## Plot predictions
for c in crossing_idx:
    plt.axvline(df.index[c],ymax=12,linestyle='--',color='g')
plt.plot([],[],linestyle='--',color='g',label = 'predicted locations')

### Plot ground truth
idx = df[df.machine_status == 'BROKEN'].index
for x in idx[:-3]:
    plt.axvline(x,ymax=12,linestyle='--',color='magenta')
#dummy plot()
plt.plot([],[],linestyle='--',color='magenta',label = 'groundtruth')

plt.legend()


### filtered version:

In [ ]:
## Split train / test
split_idx = df.shape[0] // 2 

sensor_name = 'sensor_04'
signal = df[sensor_name].copy().fillna(0)
signal -= signal.mean()
signal = filter_signal(signal)
signal = StandardScaler().fit_transform(signal.reshape(-1,1))
signal = pd.Series(signal.ravel()).fillna(0)

plt.figure(figsize=(10,8))

plt.plot(df.index[:split_idx],signal.iloc[:split_idx],label = sensor_name)

## Plot threshold
sigma_train = signal.std()
threshold_factor = 1
plt.hlines(-threshold_factor*sigma_train, df.iloc[:split_idx].index[0], df.iloc[:split_idx].index[-1], colors='r', linestyles='--',label = 'threshhold')


## Detect threshold crossings:
signal_crossing = signal.iloc[:split_idx] + threshold_factor*sigma_train 
crossing_idx = ThresholdCrossings(signal_crossing)

## Plot predictions
for c in crossing_idx:
    plt.axvline(df.index[c],ymax=12,linestyle='--',color='g')
plt.plot([],[],linestyle='--',color='g',label = 'predicted locations')

### Plot ground truth
idx = df[df.machine_status == 'BROKEN'].index
for x in idx[:-3]:
    plt.axvline(x,ymax=12,linestyle='--',color='magenta')
#dummy plot()
plt.plot([],[],linestyle='--',color='magenta',label = 'groundtruth')

plt.legend()


## Test Phase:

In [ ]:
plt.figure(figsize=(10,8))

plt.plot(df.index[split_idx:],signal.iloc[split_idx:],label = sensor_name)

## Plot threshold
plt.hlines(-threshold_factor*sigma_train, df.iloc[split_idx:].index[0], df.iloc[split_idx:].index[-1], colors='r', linestyles='--',label = 'threshhold')

## Detect threshold crossings:
signal_crossing = signal.iloc[split_idx:] + threshold_factor*sigma_train 
crossing_idx = ThresholdCrossings(signal_crossing)

## Plot predictions
for c in crossing_idx:
    plt.axvline(df.iloc[split_idx:].index[c],ymax=12,linestyle='--',color='g')
plt.plot([],[],linestyle='--',color='g',label = 'predicted locations')

### Plot ground truth
idx = df[df.machine_status == 'BROKEN'].index
for x in idx[4:]:
    plt.axvline(x,ymax=12,linestyle='--',color='magenta')
#dummy plot()
plt.plot([],[],linestyle='--',color='magenta',label = 'groundtruth')

plt.legend()


## TODO:

    - quantify predictions "accuracy"
    - Something similar to F1? Or Recall (minimize false negatives)
    - improve threshold? dynamic thresholding by incorporating more sensor data?

# Further ideas:

 - Filter signals
 - Use more of the sensors' data
 - Broken status seems to coincide with swings in signal values in many of the sensors. Use this for feature generation and then somhow try and aggregate them together.


## Some questions:

 - How accurate in time do the predictions need to be? is it okay to be 5/15/30 minutes late? 

# Hidden Markov Model:

In [ ]:
sensor_name = 'sensor_04'
signal = df[[sensor_name]].copy().fillna(0)#.rolling(3).mean().ffill().bfill()
signal = signal.values

model = hmm.GaussianHMM(n_components=2, n_iter=1000,random_state=31415,tol=1e-6)
model.fit(signal)


le = LabelEncoder()
labels = df.machine_status.copy()
labels = np.where(labels == 'RECOVERING' ,'BROKEN',labels)
labels = le.fit_transform(labels)
print(le.classes_)
print(np.unique(labels,return_counts=True))

eps = 1e-300
# model.transmat_ = np.array([[1-eps ,eps ,eps],[eps,1-eps,eps],[eps,eps,1-eps]])
model.transmat_ = np.array([[1-eps ,eps ],[eps,1-eps]])
logprob,states = model.decode(signal)
print('unique values: ',np.unique(states,return_counts=True))


#scaling for plotting:
scaled_signal = MinMaxScaler().fit_transform(signal.reshape(-1,1))
scaled_labels = MinMaxScaler().fit_transform(labels.reshape(-1,1))
scaled_states = MinMaxScaler().fit_transform(states.reshape(-1,1))

fig,ax = plt.subplots(1,1,figsize = (13,5))
ax.plot(scaled_signal,label = 'signal')
ax.plot(scaled_labels,'r-', alpha=0.6, label = 'labels')
ax.plot(scaled_states,'--', alpha=0.6, label = 'hidden states')
ax.legend()
fig.show()

# Misc

In [ ]:
## TEsting zero crossing detection:

a = [1, 2, 1, 1, -3, -4, 7, 8, 9, 10, -2, 1, -3, 5, 6, 7, -10]

a_sign_diff = np.diff(np.sign(a))
#zero crossings, only checking for positive to negative
crossings = np.where(a_sign_diff < 0,a_sign_diff,0)
crossing_idx = crossings.nonzero()

print('sign:', np.sign(a))
print('sign diff:', np.diff(np.sign(a)))
print('crossings:', crossings,)

print('where', np.where(np.diff(np.sign(a))))
print('crossover indices', crossing_idx ) 


# zero_crossings = numpy.where(numpy.diff(numpy.sign(a)))[0]
